In [277]:
import numpy as np #Importing the libraries
import pandas as pd

In [278]:
dataset = pd.read_csv('penguins.csv') #Reading the file

In [279]:
dataset.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [280]:
dataset

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [281]:
dataset.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,342.000000,342.000000,342.000000,342.000000,344.000000
mean,43.921930,17.151170,200.915205,4201.754386,2008.029070
std,5.459584,1.974793,14.061714,801.954536,0.818356
min,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,39.225000,15.600000,190.000000,3550.000000,2007.000000
50%,44.450000,17.300000,197.000000,4050.000000,2008.000000
75%,48.500000,18.700000,213.000000,4750.000000,2009.000000
max,59.600000,21.500000,231.000000,6300.000000,2009.000000


In [282]:
columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'] #Replacing the columns with the mean
for x in columns:
    dataset[x].fillna(dataset[x].mean(), inplace=True)
dataset['sex'].fillna(dataset['sex'].mode()[0], inplace=True)

In [283]:
from numpy.lib.shape_base import column_stack
column=['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g','year'] #Declaring Columns

In [284]:
def scale(data): #Scaling the dataset
    minimum = data.min()
    maximum = data.max()
    scaled_data = (data - minimum) / (maximum - minimum)
    return scaled_data

colns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
for y in colns:
    dataset[y] = scale(dataset[y])

In [285]:
dataset.shape

(344, 8)

In [286]:
dataset.loc[dataset["sex"] == "male", "sex"]   = 1 #Categorical Values
dataset.loc[dataset["sex"] == "female", "sex"] = 0 #Categorical Values

In [287]:
Y_target=dataset[['sex']] #Declaring y_target

In [288]:
data=dataset[['species', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g', 'island', 'year']]

In [289]:
data=pd.get_dummies(data) #Getting dummy values


In [290]:
training=int(len(data)*0.8)

In [291]:
training_X = data[:training] #Matrices of Features for training set
testing_X = data[training:] #Matrices of Features for testing set

training_y = Y_target[:training]#Labels for training set
testing_y = Y_target[training:]#Labels for testing set

In [292]:
import random

In [293]:
random.seed(289) #Random seed value

In [294]:
class LogitRegression(): #Logistic Regression Implementation
    def __init__(self, learning_rate, iterations, weights):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = weights
        self.bias = 0
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def cost(self, y_hat, y):
        return -np.mean(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
    
    def gradient_descent(self, X, y, y_hat):
        dW = np.dot(X.T, (y_hat-y)) / len(y)
        db = np.mean(y_hat-y)
        return dW, db
    
    def fit(self, X, y):
        self.weights = np.random.uniform(0, 1, X.shape[1])
        self.bias = 0
        self.loss = []
        for i in range(self.iterations):
            z = np.dot(X, self.weights) + self.bias
            y_hat = self.sigmoid(z.astype('float32'))
            loss = self.cost(y_hat, y)
            self.loss.append(loss)
            dW, db = self.gradient_descent(X, y, y_hat)
            self.weights = self.weights - self.learning_rate * dW
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        y_hat = self.sigmoid(z.astype('float32'))
        return (y_hat >= 0.5).astype(int)
    


In [295]:
regressor = LogitRegression(learning_rate=0.01, iterations=8000, weights=None) #Creating the model

In [296]:
regressor.fit(np.array(training_X), np.array(training_y['sex'])) #Fitting the Logistic Regression model on to the dataset

<ipython-input-294-827b374da647>:12: RuntimeWarning: divide by zero encountered in log
  return -np.mean(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
<ipython-input-294-827b374da647>:12: RuntimeWarning: invalid value encountered in multiply
  return -np.mean(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
<ipython-input-294-827b374da647>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [297]:
regressor.weights #Weights

array([2.96484732558708, 4.1841775626363225, 2.227073682876374,
       4.491293307010871, -0.12484001285753621, 0.23231808414931188,
       0.22578403866276053, 0.9351722251727246, 0.2169180509503441,
       0.14171974794609368, 0.616937309478891], dtype=object)

In [298]:
y_pred = regressor.predict(testing_X) #Using the predict method on the testing set

<ipython-input-294-827b374da647>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [299]:
y = [val for val in testing_y['sex']]

In [300]:
accuracy = np.mean(y_pred == y)#Calculating the accuracy using numpy

In [301]:
print(accuracy) #Accuracy

0.4927536231884058


In [304]:
print(accuracy/len(y_pred)*100) 

0.7141356857802983


In [303]:
import pickle

pickle.dump(regressor.weights,open("simhavis_assignment1_part1.pickle", 'wb'))